In [1]:
from glob import glob

import cv2
import numpy as np
from tqdm import tqdm

In [2]:
# Parameters for Shi-Tomasi corner detection
corner_detection_params = dict(maxCorners=300, qualityLevel=0.2, minDistance=2, blockSize=7)

In [3]:
# Parameters for Lucas-Kanade optical flow
optical_flow_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [5]:
# Variable for color to draw optical flow track
color = (0, 0, 255)

# Get all video files
files = sorted(glob('video/*'))

# Create a window
window_name = "Optical Flow"

In [6]:
for file in tqdm(files):
    # Create a VideoCapture object
    cap = cv2.VideoCapture(file)

    # Get the first frame
    ret, first_frame = cap.read()
    cv2.imshow(window_name, first_frame)
    cv2.moveWindow(window_name, 1, 1)
    if not ret: continue

    # Resize the frame
    first_frame = cv2.resize(first_frame, (640, 480))

    # Converts frame to grayscale because
    # we only need the luminance channel for detecting edges - less computationally expensive
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

    # Finds the strongest corners in the first frame by Shi-Tomasi method - we will track the optical flow for these corners
    # https://docs.opencv.org/3.0-beta/modules/imgproc/doc/feature_detection.html#goodfeaturestotrack
    prev = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_detection_params)
    
    # Creates an image filled with zero intensities with the same dimensions as the frame - for later drawing purposes
    mask = np.zeros_like(first_frame)
    idx = 0
    rgbs = {}
    while(cap.isOpened()):
        # ret = a boolean return value from getting the frame, frame = the current frame being projected in the video
        ret, frame = cap.read()
        if not ret: break

        # Resize the frame
        frame = cv2.resize(frame, (640, 480))

        # Converts each frame to grayscale - we previously only converted the first frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculates sparse optical flow by Lucas-Kanade method
        # https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowpyrlk
        prev = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_detection_params)
        next, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev, None, **optical_flow_params)

        # Selects good feature points for previous position
        good_old = prev[status == 1].astype(int)

        # Selects good feature points for next position
        good_new = next[status == 1].astype(int)

        # Draws the optical flow tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            # Returns a contiguous flattened array as (x, y) coordinates for new point
            a, b = new.ravel()

            # Returns a contiguous flattened array as (x, y) coordinates for old point
            c, d = old.ravel()

            # Draws line between new and old position with green color and 2 thickness
            mask = cv2.line(mask, (a, b), (c, d), color, 2)

            # Draws filled circle (thickness of -1) at new position with green color and radius of 3
            frame = cv2.circle(frame, (a, b), 2, color, -1)

        # Overlays the optical flow tracks on the original frame
        output = cv2.add(frame, mask)

        # Updates previous frame
        prev_gray = gray.copy()

        # Updates previous good feature points
        prev = good_new.reshape(-1, 1, 2)

        # Opens a new window and displays the output frame
        cv2.imshow(window_name, output)

        # Frames are read by intervals of 10 milliseconds.
        # The programs breaks out of the while loop when the user presses the 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # The following frees up resources and closes all windows
    cap.release()
    cv2.destroyAllWindows()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:04<00:00, 10.74s/it]
